<a href="https://colab.research.google.com/github/majid-lagzian/MMA-program/blob/main/Zindi_Loan_Default_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zindi Competition- Loan Default Prediction

In [2]:
#First we import the required libraries
#Also, we import our data from the appropriate file
import pandas as pd
import os.path as osp
import numpy as np
from datetime import timedelta

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

trainperf_path = osp.join(osp.curdir,'Zindi','trainperf.csv')
trainperf_data = pd.read_csv(trainperf_path)
trainperf_data.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,referredby,good_bad_flag
0,8a2a81a74ce8c05d014cfb32a0da1049,301994762,12,2017-07-25 08:22:56.000000,2017-07-25 07:22:47.000000,30000.0,34500.0,30,NaN,Good
1,8a85886e54beabf90154c0a29ae757c0,301965204,2,2017-07-05 17:04:41.000000,2017-07-05 16:04:18.000000,15000.0,17250.0,30,NaN,Good
2,8a8588f35438fe12015444567666018e,301966580,7,2017-07-06 14:52:57.000000,2017-07-06 13:52:51.000000,20000.0,22250.0,15,NaN,Good
3,8a85890754145ace015429211b513e16,301999343,3,2017-07-27 19:00:41.000000,2017-07-27 18:00:35.000000,10000.0,11500.0,15,NaN,Good
4,8a858970548359cc0154883481981866,301962360,9,2017-07-03 23:42:45.000000,2017-07-03 22:42:39.000000,40000.0,44000.0,30,NaN,Good


In [3]:
traindemo_path = osp.join(osp.curdir,'Zindi','traindemographics.csv')
traindemo_data = pd.read_csv(traindemo_path)
traindemo_data.head()

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients
0,8a858e135cb22031015cbafc76964ebd,1973-10-10 00:00:00.000000,Savings,3.319219,6.528604,GT Bank,NaN,NaN,NaN
1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21 00:00:00.000000,Savings,3.325598,7.119403,Sterling Bank,NaN,Permanent,NaN
2,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,NaN,NaN
3,8a858efd5ca70688015cabd1f1e94b55,1991-07-19 00:00:00.000000,Savings,3.362850,6.642485,GT Bank,NaN,Permanent,NaN
4,8a858e785acd3412015acd48f4920d04,1982-11-22 00:00:00.000000,Savings,8.455332,11.971410,GT Bank,NaN,Permanent,NaN


In [4]:
trainprev_path = osp.join(osp.curdir,'Zindi','trainprevloans.csv')
trainprev_data = pd.read_csv(trainprev_path)
trainprev_data.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,closeddate,referredby,firstduedate,firstrepaiddate
0,8a2a81a74ce8c05d014cfb32a0da1049,301682320,2,2016-08-15 18:22:40.000000,2016-08-15 17:22:32.000000,10000.0,13000.0,30,2016-09-01 16:06:48.000000,NaN,2016-09-14 00:00:00.000000,2016-09-01 15:51:43.000000
1,8a2a81a74ce8c05d014cfb32a0da1049,301883808,9,2017-04-28 18:39:07.000000,2017-04-28 17:38:53.000000,10000.0,13000.0,30,2017-05-28 14:44:49.000000,NaN,2017-05-30 00:00:00.000000,2017-05-26 00:00:00.000000
2,8a2a81a74ce8c05d014cfb32a0da1049,301831714,8,2017-03-05 10:56:25.000000,2017-03-05 09:56:19.000000,20000.0,23800.0,30,2017-04-26 22:18:56.000000,NaN,2017-04-04 00:00:00.000000,2017-04-26 22:03:47.000000
3,8a8588f35438fe12015444567666018e,301861541,5,2017-04-09 18:25:55.000000,2017-04-09 17:25:42.000000,10000.0,11500.0,15,2017-04-24 01:35:52.000000,NaN,2017-04-24 00:00:00.000000,2017-04-24 00:48:43.000000
4,8a85890754145ace015429211b513e16,301941754,2,2017-06-17 09:29:57.000000,2017-06-17 08:29:50.000000,10000.0,11500.0,15,2017-07-14 21:18:43.000000,NaN,2017-07-03 00:00:00.000000,2017-07-14 21:08:35.000000


In [5]:
trainprev_data.drop_duplicates(inplace=True)
trainprev_data.describe(include='all')

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,closeddate,referredby,firstduedate,firstrepaiddate
count,18183,1.818300e+04,18183.000000,18183,18183,18183.000000,18183.000000,18183.000000,18183,1026,18183,18183
unique,4359,NaN,NaN,18172,18173,NaN,NaN,NaN,17955,521,363,18011
top,8a858f7d5578012a01557ea194d94948,NaN,NaN,2017-04-20 14:00:44.000000,2017-06-12 10:38:36.000000,NaN,NaN,NaN,2016-11-07 00:53:42.000000,8a858fc55b2548dd015b286e452c678c,2017-07-03 00:00:00.000000,2016-08-04 00:00:00.000000
freq,26,NaN,NaN,2,2,NaN,NaN,NaN,5,14,398,7
mean,NaN,3.018395e+08,4.189353,NaN,NaN,16501.237420,19573.202931,26.692790,NaN,NaN,NaN,NaN
std,NaN,9.367767e+04,3.249490,NaN,NaN,9320.547516,10454.245277,10.946556,NaN,NaN,NaN,NaN
min,NaN,3.016001e+08,1.000000,NaN,NaN,3000.000000,3450.000000,15.000000,NaN,NaN,NaN,NaN
25%,NaN,3.017766e+08,2.000000,NaN,NaN,10000.000000,11500.000000,15.000000,NaN,NaN,NaN,NaN
50%,NaN,3.018550e+08,3.000000,NaN,NaN,10000.000000,13000.000000,30.000000,NaN,NaN,NaN,NaN
75%,NaN,3.019197e+08,6.000000,NaN,NaN,20000.000000,24500.000000,30.000000,NaN,NaN,NaN,NaN


In [6]:
trainprev_data['approveddate'] = pd.to_datetime(trainprev_data['approveddate'])
trainprev_data['creationdate'] = pd.to_datetime(trainprev_data['creationdate'])
trainprev_data['closeddate'] = pd.to_datetime(trainprev_data['closeddate'])
trainprev_data['firstduedate'] = pd.to_datetime(trainprev_data['firstduedate'])
trainprev_data['firstrepaiddate'] = pd.to_datetime(trainprev_data['firstrepaiddate'])


print(trainprev_data.dtypes)


customerid                 object
systemloanid                int64
loannumber                  int64
approveddate       datetime64[ns]
creationdate       datetime64[ns]
loanamount                float64
totaldue                  float64
termdays                    int64
closeddate         datetime64[ns]
referredby                 object
firstduedate       datetime64[ns]
firstrepaiddate    datetime64[ns]
dtype: object


In [7]:
trainprev_data['goodflag'] = trainprev_data.apply(lambda loan: 0 if ((loan['firstrepaiddate'] > loan['firstduedate']) or (loan['closeddate'] > loan['creationdate']+timedelta(days=loan['termdays']))) else 1, axis=1)

In [8]:
new_trainprev=trainprev_data.groupby('customerid').agg({'loanamount':'sum','goodflag':'sum'})

In [9]:
new_trainprev.head()

,loanamount,goodflag
customerid,,
8a1088a0484472eb01484669e3ce4e0b,10000.0,0
8a1a1e7e4f707f8b014f797718316cad,70000.0,2
8a1a32fc49b632520149c3b8fdf85139,90000.0,3
8a1eb5ba49a682300149c3c068b806c7,130000.0,6
8a1edbf14734127f0147356fdb1b1eb2,20000.0,1


In [10]:
new_trainprev.describe()

,loanamount,goodflag
count,4359.000000,4359.000000
mean,68832.759807,2.558614
std,88978.730980,2.803442
min,5000.000000,0.000000
25%,10000.000000,1.000000
50%,30000.000000,1.000000
75%,90000.000000,4.000000
max,770000.000000,25.000000


In [11]:
traindemo_data.drop_duplicates(inplace=True)
traindemo_data['birthdate'] = pd.to_datetime(traindemo_data['birthdate'])
traindemo_data.describe(include='all')

<ipython-input-11-02b5af628f85>:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  traindemo_data.describe(include='all')


,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients
count,4334,4334,4334,4334.000000,4334.000000,4334,51,3686,586
unique,4334,3297,3,NaN,NaN,18,45,6,4
top,8a858e135cb22031015cbafc76964ebd,1983-10-19 00:00:00,Savings,NaN,NaN,GT Bank,OGBA,Permanent,Graduate
freq,1,5,3414,NaN,NaN,1594,3,3134,420
first,NaN,1961-10-13 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,1996-03-28 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,4.624000,7.249975,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,7.194031,3.056594,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,-118.247009,-33.868818,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,3.354857,6.470610,NaN,NaN,NaN,NaN


In [12]:
trainperf_data.drop_duplicates(inplace=True)
trainperf_data['approveddate'] = pd.to_datetime(trainperf_data['approveddate'])
trainperf_data['creationdate'] = pd.to_datetime(trainperf_data['creationdate'])
trainperf_data.describe(include='all')

<ipython-input-12-aaf0f60ecaba>:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  trainperf_data.describe(include='all')
<ipython-input-12-aaf0f60ecaba>:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  trainperf_data.describe(include='all')


,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,referredby,good_bad_flag
count,4368,4.368000e+03,4368.000000,4368,4368,4368.000000,4368.000000,4368.000000,587,4368
unique,4368,NaN,NaN,4362,4364,NaN,NaN,NaN,521,2
top,8a2a81a74ce8c05d014cfb32a0da1049,NaN,NaN,2017-07-24 15:06:11,2017-07-05 13:28:44,NaN,NaN,NaN,8a858fc55b2548dd015b286e452c678c,Good
freq,1,NaN,NaN,2,2,NaN,NaN,NaN,8,3416
first,NaN,NaN,NaN,2017-07-01 01:35:26,2017-07-01 00:35:20,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,2017-07-30 22:55:51,2017-07-30 21:55:43,NaN,NaN,NaN,NaN,NaN
mean,NaN,3.019810e+08,5.172390,NaN,NaN,17809.065934,21257.377679,29.261676,NaN,NaN
std,NaN,1.343115e+04,3.653569,NaN,NaN,10749.694571,11943.510416,11.512519,NaN,NaN
min,NaN,3.019585e+08,2.000000,NaN,NaN,10000.000000,10000.000000,15.000000,NaN,NaN
25%,NaN,3.019691e+08,2.000000,NaN,NaN,10000.000000,13000.000000,30.000000,NaN,NaN


In [13]:
merged_df = trainperf_data.merge(new_trainprev, how='left', on='customerid')
merged_df.rename(columns={'loanamount_y':'prevloans','goodflag':'prevgoods'},inplace=True)
merged_df[['prevloans','prevgoods']]=merged_df[['prevloans','prevgoods']].fillna(0)
merged_df.describe(include='all')

<ipython-input-13-821b4261e18b>:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  merged_df.describe(include='all')
<ipython-input-13-821b4261e18b>:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  merged_df.describe(include='all')


,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount_x,totaldue,termdays,referredby,good_bad_flag,prevloans,prevgoods
count,4368,4.368000e+03,4368.000000,4368,4368,4368.000000,4368.000000,4368.000000,587,4368,4368.000000,4368.000000
unique,4368,NaN,NaN,4362,4364,NaN,NaN,NaN,521,2,NaN,NaN
top,8a2a81a74ce8c05d014cfb32a0da1049,NaN,NaN,2017-07-24 15:06:11,2017-07-05 13:28:44,NaN,NaN,NaN,8a858fc55b2548dd015b286e452c678c,Good,NaN,NaN
freq,1,NaN,NaN,2,2,NaN,NaN,NaN,8,3416,NaN,NaN
first,NaN,NaN,NaN,2017-07-01 01:35:26,2017-07-01 00:35:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,2017-07-30 22:55:51,2017-07-30 21:55:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,3.019810e+08,5.172390,NaN,NaN,17809.065934,21257.377679,29.261676,NaN,NaN,68690.934066,2.553342
std,NaN,1.343115e+04,3.653569,NaN,NaN,10749.694571,11943.510416,11.512519,NaN,NaN,88941.791429,2.802955
min,NaN,3.019585e+08,2.000000,NaN,NaN,10000.000000,10000.000000,15.000000,NaN,NaN,0.000000,0.000000
25%,NaN,3.019691e+08,2.000000,NaN,NaN,10000.000000,13000.000000,30.000000,NaN,NaN,10000.000000,1.000000


In [14]:
dummies = pd.get_dummies(merged_df[['good_bad_flag']])
merged_df = pd.concat([merged_df, dummies], axis=1)
merged_df['referredby'] = merged_df.apply(lambda ref: 0 if pd.isna(ref['referredby'])  else 1, axis=1)
merged_df.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount_x,totaldue,termdays,referredby,good_bad_flag,prevloans,prevgoods,good_bad_flag_Bad,good_bad_flag_Good
0,8a2a81a74ce8c05d014cfb32a0da1049,301994762,12,2017-07-25 08:22:56,2017-07-25 07:22:47,30000.0,34500.0,30,0,Good,200000.0,7.0,0,1
1,8a85886e54beabf90154c0a29ae757c0,301965204,2,2017-07-05 17:04:41,2017-07-05 16:04:18,15000.0,17250.0,30,0,Good,0.0,0.0,0,1
2,8a8588f35438fe12015444567666018e,301966580,7,2017-07-06 14:52:57,2017-07-06 13:52:51,20000.0,22250.0,15,0,Good,60000.0,3.0,0,1
3,8a85890754145ace015429211b513e16,301999343,3,2017-07-27 19:00:41,2017-07-27 18:00:35,10000.0,11500.0,15,0,Good,20000.0,0.0,0,1
4,8a858970548359cc0154883481981866,301962360,9,2017-07-03 23:42:45,2017-07-03 22:42:39,40000.0,44000.0,30,0,Good,150000.0,8.0,0,1


In [15]:
merged_df.columns

Index(['customerid', 'systemloanid', 'loannumber', 'approveddate',
       'creationdate', 'loanamount_x', 'totaldue', 'termdays', 'referredby',
       'good_bad_flag', 'prevloans', 'prevgoods', 'good_bad_flag_Bad',
       'good_bad_flag_Good'],
      dtype='object')

In [16]:
train_df=merged_df[['customerid', 'loannumber', 'loanamount_x', 'totaldue', 'termdays', 'referredby',
      'prevloans', 'prevgoods', 'good_bad_flag_Good']]
train_df.head()

,customerid,loannumber,loanamount_x,totaldue,termdays,referredby,prevloans,prevgoods,good_bad_flag_Good
0,8a2a81a74ce8c05d014cfb32a0da1049,12,30000.0,34500.0,30,0,200000.0,7.0,1
1,8a85886e54beabf90154c0a29ae757c0,2,15000.0,17250.0,30,0,0.0,0.0,1
2,8a8588f35438fe12015444567666018e,7,20000.0,22250.0,15,0,60000.0,3.0,1
3,8a85890754145ace015429211b513e16,3,10000.0,11500.0,15,0,20000.0,0.0,1
4,8a858970548359cc0154883481981866,9,40000.0,44000.0,30,0,150000.0,8.0,1


In [17]:
train_df.rename(columns={'loanamount_x': 'loanamount', 'good_bad_flag_Good':'goodflag'}, inplace=True)
train_df.head()

<ipython-input-17-5d35438736f5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.rename(columns={'loanamount_x': 'loanamount', 'good_bad_flag_Good':'goodflag'}, inplace=True)


,customerid,loannumber,loanamount,totaldue,termdays,referredby,prevloans,prevgoods,goodflag
0,8a2a81a74ce8c05d014cfb32a0da1049,12,30000.0,34500.0,30,0,200000.0,7.0,1
1,8a85886e54beabf90154c0a29ae757c0,2,15000.0,17250.0,30,0,0.0,0.0,1
2,8a8588f35438fe12015444567666018e,7,20000.0,22250.0,15,0,60000.0,3.0,1
3,8a85890754145ace015429211b513e16,3,10000.0,11500.0,15,0,20000.0,0.0,1
4,8a858970548359cc0154883481981866,9,40000.0,44000.0,30,0,150000.0,8.0,1


Now we can build the logistic regression

In [18]:
train_df.head(5)

,customerid,loannumber,loanamount,totaldue,termdays,referredby,prevloans,prevgoods,goodflag
0,8a2a81a74ce8c05d014cfb32a0da1049,12,30000.0,34500.0,30,0,200000.0,7.0,1
1,8a85886e54beabf90154c0a29ae757c0,2,15000.0,17250.0,30,0,0.0,0.0,1
2,8a8588f35438fe12015444567666018e,7,20000.0,22250.0,15,0,60000.0,3.0,1
3,8a85890754145ace015429211b513e16,3,10000.0,11500.0,15,0,20000.0,0.0,1
4,8a858970548359cc0154883481981866,9,40000.0,44000.0,30,0,150000.0,8.0,1


In [19]:
X=train_df.drop(['customerid','goodflag'],axis=1)
y=train_df['goodflag']

In [61]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import f1_score, make_scorer

# Assuming train_df is your training DataFrame with features and target variable

# Perform feature selection
selector = SelectKBest(score_func=f_classif, k=5)  # Select top 7 features (adjust the value of k as needed)
X_selected = selector.fit_transform(X, y)

# Get the names of the selected features
selected_feature_indices = selector.get_support(indices=True)
selected_features = X.columns[selected_feature_indices]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Define the hyperparameter grid for tuning
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1.0, 10.0],
    'solver': ['liblinear'],
    'fit_intercept': [True, False]
    # Add more hyperparameters to tune if needed
}

# Define the scorer as the macro-average of F1 scores
scorer = make_scorer(f1_score, average='macro')

# Perform cross-validated grid search for hyperparameter tuning
cv = StratifiedKFold(n_splits=25, shuffle=True, random_state=42)
grid_search = GridSearchCV(LogisticRegression(), param_grid, scoring=scorer, cv=cv)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Fit the best model on the training set
best_model.fit(X, y)

# Use the best model for predictions
#y_pred = best_model.predict(X_val)

# Compute the macro-average of F1 scores
#f1_macro = f1_score(y_val, y_pred, average='macro')
#print("Macro-Average F1 Score:", f1_macro)


LogisticRegression(penalty='l1', solver='liblinear')

In [65]:
print("Selected Features:")
print(selected_features)


Selected Features:
Index(['loannumber', 'loanamount', 'totaldue', 'prevloans', 'prevgoods'], dtype='object')


In [66]:
trainX_path = osp.join(osp.curdir,'X_test_cleaned.csv')
X_test_cleaned= pd.read_csv(trainX_path)
X_test_cleaned.head()

,customerid,loannumber,loanamount,totaldue,termdays,referredby,prevloans,prevgoods
0,8a858899538ddb8e015390510b321f08,4,10000,12250.0,30,0,26000.0,1.0
1,8a858959537a097401537a4e316e25f7,10,40000,44000.0,30,0,215000.0,7.0
2,8a8589c253ace09b0153af6ba58f1f31,6,20000,24500.0,30,0,65000.0,2.0
3,8a858e095aae82b7015aae86ca1e030b,8,30000,34500.0,30,0,120000.0,7.0
4,8a858e225a28c713015a30db5c48383d,4,20000,24500.0,30,0,30000.0,2.0


In [69]:
test_X=X_test_cleaned[['loannumber', 'loanamount', 'totaldue','termdays','referredby',
       'prevloans', 'prevgoods']]
test_X.describe()

,loannumber,loanamount,totaldue,termdays,referredby,prevloans,prevgoods
count,1450.000000,1450.000000,1450.000000,1450.000000,1450.000000,1450.000000,1450.000000
mean,5.088966,17482.758621,20874.664069,28.810345,0.126897,66287.586207,2.442069
std,3.665210,10585.420034,11726.584630,11.074839,0.332972,89046.870346,2.826918
min,2.000000,10000.000000,10500.000000,15.000000,0.000000,0.000000,0.000000
25%,2.000000,10000.000000,13000.000000,30.000000,0.000000,10000.000000,1.000000
50%,4.000000,10000.000000,13000.000000,30.000000,0.000000,30000.000000,1.000000
75%,7.000000,20000.000000,24500.000000,30.000000,0.000000,80000.000000,3.000000
max,34.000000,60000.000000,68100.000000,90.000000,1.000000,701000.000000,31.000000


In [71]:
y_predict=best_model.predict(test_X)

In [72]:
y_predict.min()

1

In [58]:
y_hat=pd.DataFrame(y_predict)
y_hat.describe()

,0
count,1450.000000
mean,0.998621
std,0.037126
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [59]:
y_hat.to_csv('yhat.csv', index=False)

In [60]:
y_hat.sum()

0    1448
dtype: int64

In [75]:
y.describe()

count    4368.000000
mean        0.782051
std         0.412900
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: goodflag, dtype: float64